In [1]:
# %pip install accelerate # charset-normalizer  # pandas python-dotenv transformers
# %pip install --upgrade accelerate

^C
Note: you may need to restart the kernel to use updated packages.


Looking in indexes: https://download.pytorch.org/whl/cu124
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     ----------------- ---------------------- 1.8/4.1 MB 11.2 MB/s eta 0:00:01
     ---------------------------------------- 4.1/4.1 MB 10.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     ---------------------- ----------------- 2.4/4.1 MB 11.2 MB/s eta 0:00:01
     ---------------------------------------- 4.1/4.1 MB 10.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/4.1 MB ? eta -:--:--
     -- ------------------------------------- 0.3/4.1 MB ? eta -:--:--
     --------------- ------------------------ 1.6/4.1 MB 6.5 MB/s eta 0:00:01
     -------------------------------------- - 3.9/4.1 MB 9.0 MB/s eta 0:00:01
     ---------------------------------

In [1]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
from huggingface_hub import login
from dotenv import load_dotenv
from torch.utils.data import Dataset

import pandas as pd

import torch
import os

c:\Compilers\Python\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# List all available CUDA devices
for i in range(torch.cuda.device_count()):
    print(f"Device {i}: {torch.cuda.get_device_name(i)}")

Device 0: NVIDIA GeForce RTX 3060 Laptop GPU


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cuda


In [4]:
# Load environment variables from .env file
load_dotenv()

token = os.getenv("HUGGINGFACE_API_TOKEN")

login(token)

In [6]:
# model_id = "meta-llama/Llama-3.2-1B"
model_id = "gpt2-medium"
output_dir = "model/gpt2-medium-food"
enpoint_url = "../apis/model/gpt-v1"

In [7]:
pipe = pipeline(
    "text-generation",
    model=model_id, 
    torch_dtype=torch.bfloat16,
    device=device
)

# response = pipe("What is the most common eaten foodstuff in USA?", return_full_text=False, truncation=True)
response = pipe("What is the most eaten food in Algeria?")
response

c:\Compilers\Python\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dini\.cache\huggingface\hub\models--gpt2-medium. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': 'What is the most eaten food in Algeria?\n\nAlgeria produces about 40 billion eggs per year, while the national consumption is about 4 billion eggs per year, Al-Hilal, the Ministry of Economic Support and Development (NEM'}]

In [ ]:
from charset_normalizer import detect

# Read a sample of the file
with open('data/Foodex1.csv', 'rb') as file:
    raw_data = file.read()

# Detect encoding
result = detect(raw_data)
print(f"Detected encoding: {result['encoding']}")

encoding = result['encoding']

In [8]:
# FAOSTAT script
def get_most_eaten_food_in_faostat_dataset():
    faostat = pd.read_csv("data/FAOSTAT_food_consumption.csv")
    # print(faostat['Item'].unique())
    # faostat[faostat['Area'] == 'Afghanistan'].head(50)

    faostat_filtered_units = faostat[faostat['Unit'] == '1000 t']

    faostat_filtered_units = faostat_filtered_units[faostat_filtered_units['Value'] > 0]

    unique_countries = faostat_filtered_units['Area'].unique()

    for country in unique_countries:
        country_data = faostat_filtered_units[faostat_filtered_units['Area'] == country]

        most_eaten_food = country_data[country_data['Value'] == country_data['Value'].max()]['Item'].values[0]

        print(f"In {country} the most eaten food is {most_eaten_food}")

# Dishes script
def get_most_eaten_food_in_dishes_dataset():
    sentences = []

    dish = pd.read_csv("data/dishes.csv")

    unique_countries = dish['countries'].unique()
    list_of_foods = []
    for country in unique_countries:
        country_data = dish[dish['countries'] == country]

        country_regions = country_data['regions'].unique()

        for region in country_regions:
            region_data = country_data[country_data['regions'] == region]

            if len(region_data['english_name'].values) == 0:
                continue
            
            sentences.append(f"What is the most eaten food in {country}, {region}? In {country}, {region} the most eaten food is {region_data['english_name'].values[0]}")
            sentences.append(f"What is the most eaten food in {region}? In {country}, {region} the most eaten food is {region_data['english_name'].values[0]}")
            sentences.append(f"What do people in {region} eat? In {country}, {region} the most eaten food is {region_data['english_name'].values[0]}")
            sentences.append(f"What do people in {country}, {region} eat? In {country}, {region} the most eaten food is {region_data['english_name'].values[0]}")
            list_of_foods.append(f"{region_data['english_name'].values[0]}")
            # print(f"What is the most eaten food in {country}? In {country}, {region} the most eaten food is {region_data['english_name'].values[0]}")

        sentences.append(f"What is the most eaten food in {country}? The most common eaten foods in {country}: {', '.join(list_of_foods).rstrip()}")
        sentences.append(f"What is the most eaten food in {country}? The most eaten foods in {country}: {', '.join(list_of_foods).rstrip()}")
        sentences.append(f"What do people in {country} eat? The most eaten foods in {country}: {', '.join(list_of_foods).rstrip()}")
        list_of_foods = []

    return sentences

In [9]:
# Get some sentences to ask our AI
sentences = get_most_eaten_food_in_dishes_dataset()

# print(sentences)

filtered_items = [item for item in sentences if "Croatia" in item]

print(filtered_items)

['What is the most eaten food in Croatia, Continental Croatia? In Croatia, Continental Croatia the most eaten food is Poppy roll or Ma-kov-nya-cha', 'What is the most eaten food in Continental Croatia? In Croatia, Continental Croatia the most eaten food is Poppy roll or Ma-kov-nya-cha', 'What do people in Continental Croatia eat? In Croatia, Continental Croatia the most eaten food is Poppy roll or Ma-kov-nya-cha', 'What do people in Croatia, Continental Croatia eat? In Croatia, Continental Croatia the most eaten food is Poppy roll or Ma-kov-nya-cha', 'What is the most eaten food in Croatia? The most common eaten foods in Croatia: Poppy roll or Ma-kov-nya-cha', 'What is the most eaten food in Croatia? The most eaten foods in Croatia: Poppy roll or Ma-kov-nya-cha', 'What do people in Croatia eat? The most eaten foods in Croatia: Poppy roll or Ma-kov-nya-cha']


In [10]:
# Custom Dataset class for text generation
class TextGenerationDataset(Dataset):
    def __init__(self, texts, tokenizer, device, max_length=512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.max_length = max_length
        self.device = device

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Tokenize the text with padding and truncation
        encoding = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        ).to(device)
        # Labels are the same as input_ids for causal language modeling
        encoding["labels"] = encoding["input_ids"]
        return {key: val.squeeze(0) for key, val in encoding.items()}

In [11]:
def train_model(model_to_train, tokenizer_for_model, texts, save_model_dir):
    # Prepare the dataset
    train_dataset = TextGenerationDataset(texts, tokenizer_for_model, device)

    # Training arguments
    training_args = TrainingArguments(
        output_dir="trainer",               # Output directory
        learning_rate=5e-5,                 # Learning rate
        per_device_train_batch_size=2,      # Batch size
        weight_decay=0.01,                  # Weight decay
        save_strategy="no",                 # No saving on checkpoints
        logging_dir="logs",                 # Log directory
        logging_steps=10,                   # Log every 10 steps
        fp16=True,                          # Enable mixed precision (if supported)
    )

    # Initialize the Trainer
    trainer = Trainer(
        model=model_to_train,
        args=training_args,
        train_dataset=train_dataset,
        tokenizer=tokenizer_for_model,
    )

    # Fine-tune the model
    trainer.train()

    model_to_train.save_pretrained(save_model_dir)
    tokenizer_for_model.save_pretrained(save_model_dir)

In [12]:
# Load the pretrained text-generation model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id
).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Ensure padding tokens are set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Loading dataset
texts = get_most_eaten_food_in_dishes_dataset()

train_model(model, tokenizer, texts, output_dir)

C:\Users\Dini\AppData\Local\Temp\ipykernel_13976\2987876981.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  0%|          | 10/2262 [00:09<1:05:56,  1.76s/it]

{'loss': 9.8787, 'grad_norm': 175.43577575683594, 'learning_rate': 4.995579133510168e-05, 'epoch': 0.01}


  1%|          | 20/2262 [00:38<1:44:12,  2.79s/it]

{'loss': 0.8372, 'grad_norm': 1.2287193536758423, 'learning_rate': 4.9734748010610085e-05, 'epoch': 0.03}


  1%|▏         | 30/2262 [01:07<1:45:47,  2.84s/it]

...

{'loss': 0.0238, 'grad_norm': 0.43072396516799927, 'learning_rate': 4.4208664898320077e-07, 'epoch': 2.98}


100%|█████████▉| 2260/2262 [2:47:12<00:08,  4.11s/it]

{'loss': 0.0229, 'grad_norm': 0.46699124574661255, 'learning_rate': 2.2104332449160039e-07, 'epoch': 3.0}


100%|██████████| 2262/2262 [2:47:21<00:00,  4.44s/it]


{'train_runtime': 10041.1387, 'train_samples_per_second': 0.451, 'train_steps_per_second': 0.225, 'train_loss': 0.0901151361235415, 'epoch': 3.0}


In [ ]:
# Retrain the model if it has missing things
model = AutoModelForCausalLM.from_pretrained(
    output_dir
).to(device)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

# Ensure padding tokens are set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Loading dataset
texts = get_most_eaten_food_in_dishes_dataset()

train_model(model, tokenizer, texts, output_dir + "-1")

In [13]:
model2 = AutoModelForCausalLM.from_pretrained(output_dir)

In [77]:
# Fine-tuned model

classifier = pipeline(
    "text-generation", 
    model=model2,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    max_length=100,
    device=device
)

response = classifier("What is the most eaten food in Brazil?")
response

[{'generated_text': 'What is the most eaten food in Brazil? The most common eaten foods in Brazil: Cheese spaetzle, nan'}]

In [22]:
# Move model to endpoint
finished_model = AutoModelForCausalLM.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

finished_model.save_pretrained(enpoint_url)
tokenizer.save_pretrained(enpoint_url)

('../apis/model/gpt-v1\\tokenizer_config.json',
 '../apis/model/gpt-v1\\special_tokens_map.json',
 '../apis/model/gpt-v1\\vocab.json',
 '../apis/model/gpt-v1\\merges.txt',
 '../apis/model/gpt-v1\\added_tokens.json',
 '../apis/model/gpt-v1\\tokenizer.json')